# Chapter 10 - Data Aggregation and Group Operations

## Data Aggregation

In [1]:
import pandas as pd

In [2]:
# DATA PREPARATION STEP

# Read from CSV file
df = pd.read_csv('dataset-A3-loans.csv')
display(df.head(8))

,id,funded_amount,term,interest_rate,grade,employee_length,home_ownership,annual_income,purpose,title
0,721751,7000.0,36 months,14.91,D,2 years,RENT,46000.0,debt_consolidation,Debt Removal
1,40277218,16800.0,60 months,16.49,D,4 years,RENT,45500.0,home_improvement,Home improvement
2,68416017,1500.0,36 months,9.17,B,10+ years,MORTGAGE,83000.0,major_purchase,Major purchase
3,59481461,8000.0,36 months,12.29,C,2 years,RENT,74000.0,debt_consolidation,Debt consolidation
4,73003,3200.0,36 months,9.96,B,< 1 year,MORTGAGE,150000.0,other,New Bathroom
5,55917749,5000.0,36 months,12.29,C,10+ years,MORTGAGE,55000.0,home_improvement,Home improvement
6,1149328,11500.0,36 months,16.29,C,2 years,MORTGAGE,68000.0,credit_card,Credit Card
7,1614457,6000.0,36 months,15.80,C,5 years,MORTGAGE,36000.0,debt_consolidation,Debt consolidation


A summary of all the built-in aggregation functions after using `.groupby()` are:

- `count()`, `sum()`, `mean()`, `std()`, `var()`, `min()`, `max()`

In [3]:
df_by_grade_grouped = df.groupby('grade')[['funded_amount']]

In [4]:
print(df_by_grade_grouped.count())

       funded_amount
grade               
A                  6
B                 21
C                 15
D                  4
E                  3
F                  1


In [5]:
print(df_by_grade_grouped.mean())
print(df_by_grade_grouped.std())
print(df_by_grade_grouped.var())

       funded_amount
grade               
A       12083.333333
B       12517.857143
C       10670.000000
D       12568.750000
E       18683.333333
F       25000.000000
       funded_amount
grade               
A       11416.902674
B        7915.602247
C        7982.728007
D        8447.222793
E       13381.921885
F                NaN
       funded_amount
grade               
A       1.303457e+08
B       6.265676e+07
C       6.372395e+07
D       7.135557e+07
E       1.790758e+08
F                NaN


In [6]:
print(df_by_grade_grouped.min())
print(df_by_grade_grouped.max())

       funded_amount
grade               
A             4000.0
B             1500.0
C             1000.0
D             4175.0
E             4800.0
F            25000.0
       funded_amount
grade               
A            34000.0
B            35000.0
C            26000.0
D            22300.0
E            31500.0
F            25000.0


While `.quantile()` is not explicitly implemented for `groupby()`, it is a `Series` method and thus available for use. Internally, GroupBy efficiently slices up the Series, calls `chunk.quantile(0.9)` for each `chunk`, and then assembles those results together into the final output. The same goes for `.median()`.

In [7]:
df_by_term = df.groupby('term')['funded_amount']

print(df_by_term.quantile(0.25))
print(df_by_term.median())
print(df_by_term.quantile(0.75))

term
 36 months     4800.0
 60 months    17500.0
Name: funded_amount, dtype: float64
term
 36 months     6500.0
 60 months    20375.0
Name: funded_amount, dtype: float64
term
 36 months    11075.0
 60 months    22950.0
Name: funded_amount, dtype: float64


To create custom aggregation functions, pass any function that aggregates an array. Use the `.agg()` method for it.

In [8]:
def diff_top_x(s):
    std = s.sort_values()
    return std.tail(3).sum() - std.head(3).sum()

In [9]:
df.groupby('grade')['annual_income'].agg(diff_top_x)

grade
A     65000.0
B    330000.0
C    153000.0
D     63250.0
E         0.0
F         0.0
Name: annual_income, dtype: float64

Since `.describe()` is a `Series` function, it can be used too following a `.groupby()`.

In [10]:
df.groupby('grade')['annual_income'].describe()

,count,mean,std,min,25%,50%,75%,max
grade,,,,,,,,
A,6.0,81166.666667,16166.838487,67000.0,71500.0,74500.0,86500.0,110000.0
B,21.0,75089.571429,36543.053832,40000.0,45000.0,65000.0,83000.0,150000.0
C,15.0,60502.666667,20152.244635,36000.0,48520.0,55000.0,70000.0,115000.0
D,4.0,51187.500000,26574.874569,25000.0,40375.0,45750.0,56562.5,88250.0
E,3.0,91691.666667,42483.940593,46000.0,72537.5,99075.0,114537.5,130000.0
F,1.0,68000.000000,NaN,68000.0,68000.0,68000.0,68000.0,68000.0


**References:**

Python for Data Analysis, 2nd Edition, McKinney (2017)